In [26]:
import json
import numpy as np
import os
import pandas as pd

# kenya_bound = [33.89, -4.68, 41.86, 5.51] # [west, south, east, north] bounds of Kenya
# bound = [2, 4, 16, 14] # [west, south, east, north] bounds 
# bound = [-20, 0, 10, 50]
bound = [10, 0, 40, 50]
# bound = [120,30,122.25,32] # [west, south, east, north] bounds of Shanghai
save_name = f"Africa_grid_{bound[0]}_{bound[1]}_{bound[2]}_{bound[3]}"
os.makedirs(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata', exist_ok=True)

# global_grids = pd.read_csv("human_settlement_grid_cells.csv")
global_grids = pd.read_csv("human_settlement_grid_cells_africa_left.csv")
select_grids = global_grids[
    (global_grids['lon_min'] > bound[0]) &
    (global_grids['lon_max'] < bound[2]) &
    (global_grids['lat_min'] > bound[1]) &
    (global_grids['lat_max'] < bound[3])
].reset_index(drop=True)

# Create grid cells of 0.25x0.25 degree
# lon_min, lat_min, lon_max, lat_max = bound
# lons = np.arange(lon_min, lon_max + 0.25, 0.25)
# lats = np.arange(lat_min, lat_max + 0.25, 0.25)

if not os.path.exists(f'/code/GEE_download/{save_name}/{save_name}_grid_cells.json'):
    grid_dict = {}
    for i, row in select_grids.iterrows():
        lon_min = row['lon_min']
        lat_min = row['lat_min']
        lon_max = row['lon_max']
        lat_max = row['lat_max']
        cell_name = f"{save_name}_grid_{i}"
        grid_dict[cell_name] = {
            "lon_min": f"{float(lon_min):.4f}",
            "lon_max": f"{float(lon_max):.4f}",
            "lat_min": f"{float(lat_min):.4f}",
            "lat_max": f"{float(lat_max):.4f}"
        }

    # Save the grid cells as a JSON file
    with open(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}.json', 'w') as f:
        json.dump(grid_dict, f, indent=4)

    print(f"Created {len(grid_dict)} grid cells and saved to {save_name}.json")

else:
    grid_dict = json.load(open(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}.json', 'r'))
pd.DataFrame(grid_dict).T.reset_index().rename(columns={'index':'grid_name'}).to_csv(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}_grid_cells.csv', index=False)

Created 4476 grid cells and saved to Africa_grid_10_0_40_50.json


If you need search according to country boundary

In [ ]:
# import json
# import numpy as np
# import os
# import pandas as pd
# import geopandas as gpd
# from shapely.geometry import Polygon

# # 按照国家获取网格
# global_boundary = gpd.read_file("/nas/houce/country_boundary/ne_110m_admin_0_countries.geojson")

# # bound = [120,30,122.25,32] # [west, south, east, north] bounds of Shanghai
# save_name = f"GB_grid"
# os.makedirs(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata', exist_ok=True)

# # global_grids = pd.read_csv("human_settlement_grid_cells.csv")
# global_grids = pd.read_csv("human_settlement_grid_cells.csv")
# global_grids['geometry'] = global_grids.apply(
#     lambda row: Polygon([
#         (row['lon_min'], row['lat_min']),
#         (row['lon_min'], row['lat_max']),
#         (row['lon_max'], row['lat_max']),
#         (row['lon_max'], row['lat_min']),
#         (row['lon_min'], row['lat_min'])
#     ]), axis=1
# )
# global_grids_gdf = gpd.GeoDataFrame(global_grids, geometry='geometry', crs="EPSG:4326")
# select_grids = global_grids_gdf.sjoin(global_boundary[global_boundary['ADMIN'].str.contains('United Kingdom')], how='inner', predicate='intersects')

# if not os.path.exists(f'/code/GEE_download/{save_name}/{save_name}_grid_cells.json'):
#     grid_dict = {}
#     for i, row in select_grids.iterrows():
#         lon_min = row['lon_min']
#         lat_min = row['lat_min']
#         lon_max = row['lon_max']
#         lat_max = row['lat_max']
#         cell_name = f"{save_name}_grid_{i}"
#         grid_dict[cell_name] = {
#             "lon_min": f"{float(lon_min):.4f}",
#             "lon_max": f"{float(lon_max):.4f}",
#             "lat_min": f"{float(lat_min):.4f}",
#             "lat_max": f"{float(lat_max):.4f}"
#         }

#     # Save the grid cells as a JSON file
#     with open(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}.json', 'w') as f:
#         json.dump(grid_dict, f, indent=4)

#     print(f"Created {len(grid_dict)} grid cells and saved to {save_name}.json")

# else:
#     grid_dict = json.load(open(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}.json', 'r'))
# pd.DataFrame(grid_dict).T.reset_index().rename(columns={'index':'grid_name'}).to_csv(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}_grid_cells.csv', index=False)


Created 531 grid cells and saved to GB_grid.json


In [ ]:
# def generate_code_script(batch_size, save_name, grid_dict, mode='python'):
#     regions = []
#     for cell_name, coords in grid_dict.items():
#         region = {
#             'name': cell_name,
#             'geometry': f"ee.Geometry.Rectangle([{coords['lon_min']}, {coords['lat_min']}, {coords['lon_max']}, {coords['lat_max']}])"
#         }
#         regions.append(region)

#     if mode == "GEE":
#         gee_code = """// =================================================================================
#         // **第二步：设置通用的参数**
#         // =================================================================================

#         // 加载影像集合
#         var dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL');

#         // 开始和结束时间
#         var date_s = '2023-01-01';
#         var date_e = '2024-01-01';

#         // 下载分辨率
#         var resolution = 10;

#         // 可视化参数
#         var visParams = {min: -0.3, max: 0.3, bands: ['A01', 'A16', 'A09']};


#         // =================================================================================
#         // **第三步：循环遍历每个区域并创建导出任务**
#         // =================================================================================

#         // 使用 forEach 循环来处理 'regions' 列表中的每一个元素
#         regions.forEach(function(region) {

#             // 1. 筛选影像
#             // 注意：这里的 filterBounds 使用的是当前循环中的 region.geometry
#             var image = dataset
#             .filterDate(date_s, date_e)
#             .filterBounds(region.geometry)
#             .first();

#             // 2. 在地图上显示影像（可选，方便检查）
#             // 为了区分，我们用区域的名字来命名图层
#             Map.addLayer(image, visParams, region.name + ' embeddings');
#             Map.centerObject(region.geometry, 10); // 将地图中心移动到当前区域

#             // 3. 为当前区域创建导出任务
#             Export.image.toDrive({
#             image: image,
#             // 在描述和文件名中加入区域名称，以保证每个任务和文件都独一无二
#             description: 'Satellite_Embedding_' + region.name,
#             folder: 'GEE_Exports', // Google Drive 中的文件夹
#             fileNamePrefix: 'Satellite_Embedding_' + region.name, // 文件名前缀
#             scale: resolution,
#             region: region.geometry, // 使用当前循环中的区域范围
#             maxPixels: 1e13
#             });
#         });"""


#         # Format the output in JavaScript syntax for Earth Engine
#         js_regions = "var regions = [\n"
#         for region in regions:
#             js_regions += f"  {{\n"
#             js_regions += f"    name: '{region['name']}',\n"
#             js_regions += f"    geometry: {region['geometry']}\n"
#             js_regions += f"  }},\n"
#         js_regions += "];"

        
#         batched_regions = []
#         for i in range(0, len(regions), batch_size):
#             batch = regions[i:i+batch_size]
#             batched_regions.append(batch)

#         # Create separate JavaScript files for each batch
#         for batch_idx, batch in enumerate(batched_regions):
#             batch_js = "var regions = [\n"
#             for region in batch:
#                 batch_js += f"  {{\n"
#                 batch_js += f"    name: '{region['name']}',\n"
#                 batch_js += f"    geometry: {region['geometry']}\n"
#                 batch_js += f"  }},\n"
#             batch_js += "];" + "\n" + "\n" + gee_code

#             # Create the directory if it doesn't exist
#             os.makedirs(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}_download_GEE', exist_ok=True)
#             # Save each batch to a separate file
#             with open(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}_download_GEE/{save_name}_regions_batch_{batch_idx+1}.txt', 'w') as f:
#                 f.write(batch_js)

#             print(f"Batch {batch_idx+1} with {len(batch)} regions saved to {save_name}_regions_batch_{batch_idx+1}.txt")
#             print(f"Generated {len(regions)} regions for Earth Engine")


#     elif mode=="python":
#         js_regions = "var regions = [\n"
#         for region in regions:
#             js_regions += f"  {{\n"
#             js_regions += f"    'name': {region['name']}',\n"
#             js_regions += f"    'geometry': {region['geometry']}\n"
#             js_regions += f"  }},\n"
#         js_regions += "]"


#         # Split regions into groups of 25
#         batched_regions = []
#         for i in range(0, len(regions), batch_size):
#             batch = regions[i:i+batch_size]
#             batched_regions.append(batch)

#             # os.makedirs(f'/code/GEE_download/{save_name}/{save_name}_download_python/', exist_ok=True)

#             json_batch = [
#                 {
#                     "name": region["name"],
#                     "geometry": region["geometry"]
#                 }
#                 for region in batch
#             ]
#             os.makedirs(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}_download_python/", exist_ok=True)
#             with open(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}_download_python/{save_name}_regions_batch_{i//batch_size+1}.json', 'w', encoding='utf-8') as f_json:
#                 json.dump(json_batch, f_json, ensure_ascii=False, indent=4)
#             print(f"Batch {i//batch_size+1} with {len(batch)} regions saved to {save_name}_regions_batch_{i//batch_size+1}.json")

# # save_name = f"Africa_grid_{bound[0]}_{bound[1]}_{bound[2]}_{bound[3]}"
# generate_code_script(25, save_name, grid_dict, mode='python')

Batch 1 with 25 regions saved to GB_grid_regions_batch_1.json
Batch 2 with 25 regions saved to GB_grid_regions_batch_2.json
Batch 3 with 25 regions saved to GB_grid_regions_batch_3.json
Batch 4 with 25 regions saved to GB_grid_regions_batch_4.json
Batch 5 with 25 regions saved to GB_grid_regions_batch_5.json
Batch 6 with 25 regions saved to GB_grid_regions_batch_6.json
Batch 7 with 25 regions saved to GB_grid_regions_batch_7.json
Batch 8 with 25 regions saved to GB_grid_regions_batch_8.json
Batch 9 with 25 regions saved to GB_grid_regions_batch_9.json
Batch 10 with 25 regions saved to GB_grid_regions_batch_10.json
Batch 11 with 25 regions saved to GB_grid_regions_batch_11.json
Batch 12 with 25 regions saved to GB_grid_regions_batch_12.json
Batch 13 with 25 regions saved to GB_grid_regions_batch_13.json
Batch 14 with 25 regions saved to GB_grid_regions_batch_14.json
Batch 15 with 25 regions saved to GB_grid_regions_batch_15.json
Batch 16 with 25 regions saved to GB_grid_regions_batch_16

## 补全图像


In [ ]:
save_name = f"Africa_grid_{bound[0]}_{bound[1]}_{bound[2]}_{bound[3]}"
tem_na = pd.read_csv(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/tem_na.csv")

selected_keys = tem_na['grid_name'].iloc[:214].tolist()
selected_grids = {k: grid_dict[k] for k in selected_keys if k in grid_dict}

save_name_updated = f"Africa_grid_{bound[0]}_{bound[1]}_{bound[2]}_{bound[3]}_supplement_R2"
generate_code_script(25, save_name_updated, selected_grids, mode='python')

FileNotFoundError: [Errno 2] No such file or directory: '/nas/houce/Alphaearth_embedding/GEE_extracted/Africa_grid_10_0_40_50/metadata/tem_na.csv'

In [ ]:
import shutil
import os

target_savename = f"Africa_grid_{bound[0]}_{bound[1]}_{bound[2]}_{bound[3]}"

src_dir = f"/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name_updated}/metadata/{save_name_updated}_download_python/"
dst_dir = f"/nas/houce/Alphaearth_embedding/GEE_extracted/{target_savename}/metadata/{save_name_updated}_download_python"

# 如果目标目录不存在则创建
os.makedirs(dst_dir, exist_ok=True)

# 移动整个目录及其内容
# if os.path.exists(dst_dir):
#     print("already exist")
# else:
shutil.move(src_dir, dst_dir)
# print(f"已移动 {src_dir} 到 {dst_dir}")

# 如果src_dir还存在（如部分内容未移动），则递归删除
if os.path.exists(src_dir):
    shutil.rmtree(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name_updated}")
    print(f"已删除原始目录及其所有文件: {src_dir}")


In [ ]:
# # Convert Kenya grid cells to Earth Engine regions format
# regions = []

# gee_code = """// =================================================================================
# // **第二步：设置通用的参数**
# // =================================================================================

# // 加载影像集合
# var dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL');

# // 开始和结束时间
# var date_s = '2023-01-01';
# var date_e = '2024-01-01';

# // 下载分辨率
# var resolution = 10;

# // 可视化参数
# var visParams = {min: -0.3, max: 0.3, bands: ['A01', 'A16', 'A09']};


# // =================================================================================
# // **第三步：循环遍历每个区域并创建导出任务**
# // =================================================================================

# // 使用 forEach 循环来处理 'regions' 列表中的每一个元素
# regions.forEach(function(region) {

#     // 1. 筛选影像
#     // 注意：这里的 filterBounds 使用的是当前循环中的 region.geometry
#     var image = dataset
#     .filterDate(date_s, date_e)
#     .filterBounds(region.geometry)
#     .first();

#     // 2. 在地图上显示影像（可选，方便检查）
#     // 为了区分，我们用区域的名字来命名图层
#     Map.addLayer(image, visParams, region.name + ' embeddings');
#     Map.centerObject(region.geometry, 10); // 将地图中心移动到当前区域

#     // 3. 为当前区域创建导出任务
#     Export.image.toDrive({
#     image: image,
#     // 在描述和文件名中加入区域名称，以保证每个任务和文件都独一无二
#     description: 'Satellite_Embedding_' + region.name,
#     folder: 'GEE_Exports', // Google Drive 中的文件夹
#     fileNamePrefix: 'Satellite_Embedding_' + region.name, // 文件名前缀
#     scale: resolution,
#     region: region.geometry, // 使用当前循环中的区域范围
#     maxPixels: 1e13
#     });
# });"""

# for cell_name, coords in grid_dict.items():
#     region = {
#         'name': cell_name,
#         'geometry': f"ee.Geometry.Rectangle([{coords['lon_min']}, {coords['lat_min']}, {coords['lon_max']}, {coords['lat_max']}])"
#     }
#     regions.append(region)

# # Format the output in JavaScript syntax for Earth Engine
# js_regions = "var regions = [\n"
# for region in regions:
#     js_regions += f"  {{\n"
#     js_regions += f"    name: '{region['name']}',\n"
#     js_regions += f"    geometry: {region['geometry']}\n"
#     js_regions += f"  }},\n"
# js_regions += "];"

# # Split regions into groups of 25

# batch_size = 10
# batched_regions = []
# for i in range(0, len(regions), batch_size):
#     batch = regions[i:i+batch_size]
#     batched_regions.append(batch)

# # Create separate JavaScript files for each batch
# for batch_idx, batch in enumerate(batched_regions):
#     batch_js = "var regions = [\n"
#     for region in batch:
#         batch_js += f"  {{\n"
#         batch_js += f"    name: '{region['name']}',\n"
#         batch_js += f"    geometry: {region['geometry']}\n"
#         batch_js += f"  }},\n"
#     batch_js += "];" + "\n" + "\n" + gee_code

#     # Create the directory if it doesn't exist
#     os.makedirs('/Kenya', exist_ok=True)
#     # Save each batch to a separate file
#     with open(f'/code/GEE_download/Kenya/Kenya_download_GEE/kenya_regions_batch_{batch_idx+1}.txt', 'w') as f:
#         f.write(batch_js)
    
#     print(f"Batch {batch_idx+1} with {len(batch)} regions saved to kenya_regions_batch_{batch_idx+1}.txt")
# print(f"Generated {len(regions)} regions for Earth Engine")

# # # Save the JavaScript regions to a text file
# # with open('kenya_regions_for_ee.txt', 'w') as f:
# #     f.write(js_regions)

# # print(f"JavaScript regions saved to kenya_regions_for_ee.js")

In [ ]:
# # for python
# regions = []

# for cell_name, coords in grid_dict.items():
#     region = {
#         'name': cell_name,
#         'geometry': f"ee.Geometry.Rectangle([{coords['lon_min']}, {coords['lat_min']}, {coords['lon_max']}, {coords['lat_max']}])"
#     }
#     regions.append(region)

# js_regions = "var regions = [\n"


# for region in regions:
#     js_regions += f"  {{\n"
#     js_regions += f"    name: '{region['name']}',\n"
#     js_regions += f"    geometry: {region['geometry']}\n"
#     js_regions += f"  }},\n"
# js_regions += "];"


# # Split regions into groups of 25
# batch_size = 10
# batched_regions = []
# for i in range(0, len(regions), batch_size):
#     batch = regions[i:i+batch_size]
#     batched_regions.append(batch)

# # Create separate JavaScript files for each batch
# for batch_idx, batch in enumerate(batched_regions):
#     batch_js = "var regions = [\n"
#     for region in batch:
#         batch_js += f"  {{\n"
#         batch_js += f"    name: '{region['name']}',\n"
#         batch_js += f"    geometry: {region['geometry']}\n"
#         batch_js += f"  }},\n"
#     batch_js += "];"

#     # Create the directory if it doesn't exist
#     # os.makedirs('/Kenya', exist_ok=True)
#     # Save each batch to a separate file
#     with open(f'/code/GEE_download/{save_name}/{save_name}_download_py/{save_name}_regions_batch_{batch_idx+1}.txt', 'w') as f:
#         f.write(batch_js)

#     print(f"Batch {batch_idx+1} with {len(batch)} regions saved to {save_name}_regions_batch_{batch_idx+1}.txt")
# print(f"Generated {len(regions)} regions for Earth Engine")